In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [15]:
class movie_recommendation_vec:
    def __init__(self, **kargs):
        self.topn = kargs.get('topn',10)
        self.vec_type = kargs.get('vec_type','tfidf')
        self.df = kargs.get('data', pd.read_csv('./data/merged.csv'))
        self.a, self.b, self.c = kargs.get('a', 0.6), kargs.get('b', 0.2), kargs.get('c', 0.2)
        self.vote_thres = kargs.get('vote_thres', 100)
        self.verbose = kargs.get('verbose', 1)
        
        self.cvec = CountVectorizer(min_df=0, ngram_range=(1,2))
        self.stops = []
        with open('./data/total_stopwords', encoding='utf-8') as f:
            self.stops.append(f.readline()[:-2])
        
        if self.vec_type == 'tfidf':
            self.vec = TfidfVectorizer(analyzer='word', ngram_range=(1,5),stop_words=self.stops,
                       min_df=10, max_df=0.95, max_features=10000)
        elif self.vec_type == 'count':
            self.vec = CountVectorizer(analyzer='word',ngram_range=(1,1), stop_words=self.stops,
                      min_df=10, max_df=0.95, max_features=10000)
        else:
            raise ValueError('vec_type is not in [tfidf, count]')
            
        if self.verbose == 1:
            print('-'*35)
            print('# Parameters')
            print('      a, b, c        : {0}, {1}, {2}'.format(self.a, self.b, self.c))
            print('vote count threshold :', self.vote_thres)
            print('vec_type :',self.vec_type.capitalize())
            print('weighted_sum = vec_sim_scaled*{0}(a) + genre_scaled*{1}(b) + wvote_scaled*{2}(c)'.format(self.a, self.b, self.c))
            print('-'*35)
            
    def search_title(self, title_name):
        return self.df[self.df['title'].str.contains(title_name)].title
    
    def genre_sim_sorted(self, title_idx):
        genre_literal = self.df['genre'].apply(lambda x: x.replace('|',' '))
        genre = self.cvec.fit_transform(genre_literal)
        genre_sim = cosine_similarity(genre,genre)

        return np.array([(idx,sim) for idx,sim in enumerate(genre_sim[title_idx])])            

    def raw_to_vec(self, data_preprocess):
        return self.vec.fit_transform(data_preprocess)
    
    def cos_sim(self, data_preprocess, title_idx):
        cos_sims = []
        data_transformed = self.raw_to_vec(data_preprocess)
        
        for df_idx, src_idx in zip(self.df.index, range(data_transformed.shape[0])):
            if df_idx != title_idx:
                cos_sims.append((df_idx, cosine_similarity(data_transformed[title_idx].reshape(1,-1),
                                                          data_transformed[src_idx].reshape(1,-1))))
        return cos_sims
    
    def similar_vec_movies(self, title_idx):
        idx_sims = np.array(self.cos_sim(self.df['plot_preprocessed_kkma'], title_idx))
        sims_scaled = MinMaxScaler().fit_transform(idx_sims[:,1].reshape(-1,1))
        idx_sims[:,1] = sims_scaled.reshape(-1)

        idx_sims = np.array(sorted(idx_sims, key=lambda x: x[1], reverse=True))
        result_df = self.df.loc[idx_sims[:,0]]
        result_df['vec_sim'] = idx_sims[:,1]
        
        return result_df[result_df['vote_count'] > self.vote_thres]
    
    def result_by_weights(self, dataf):
        dataf['weighted_sum'] = dataf['vec_sim_scaled']*self.a + dataf['genre_scaled']*self.b + dataf['wvote_scaled']*self.c
        
        return dataf.sort_values('weighted_sum', ascending=False)
    
    def getMovies(self, title):
        # no title result
        try:title_idx = self.df[self.df['title']== title].index.values[0]
        except:
            raise ValueError('There is no such title name in data. Search with "search_title" function')

        # get movies
        result = self.similar_vec_movies(title_idx)
        
        # IMDB's weighted_vote
        def weighted_vote_average(record):
            v, r = record['vote_count'], record['rating']
            return (v/(v+m))*r + (m/(m+v))*c
        c = result['rating'].mean()
        m = result['vote_count'].quantile(.6)
        result['weighted_vote'] = result.apply(weighted_vote_average,axis=1)
        
        # merge with genre
        genre_sim = self.genre_sim_sorted(title_idx)
        Result = result.rename(columns={'key_0': 'key_00'})
        result_with_genre = pd.merge(Result, pd.Series(genre_sim[:,1], name='genre_sim'), left_on=Result.index, right_on=genre_sim[:,0],)
        
        # minmax scale
        result_with_genre['vec_sim_scaled'] = MinMaxScaler().fit_transform(result_with_genre['vec_sim'].values.reshape(-1,1))
        result_with_genre['wvote_scaled'] = MinMaxScaler().fit_transform(result_with_genre['weighted_vote'].values.reshape(-1,1))
        result_with_genre['genre_scaled'] = MinMaxScaler().fit_transform(result_with_genre['genre_sim'].values.reshape(-1,1))

        # (optional)remove data genre score is 0
        no_genre_score_idx = result_with_genre[result_with_genre['genre_sim'] == 0].index
        result_with_genre.drop(no_genre_score_idx, inplace=True)
        
        result_with_genre = self.result_by_weights(result_with_genre)
        return result_with_genre.head(self.topn)

In [16]:
recom = movie_recommendation_vec(vec_type='tfidf')

-----------------------------------
# Parameters
      a, b, c        : 0.6, 0.2, 0.2
vote count threshold : 100
vec_type : Tfidf
weighted_sum = vec_sim_scaled*0.6(a) + genre_scaled*0.2(b) + wvote_scaled*0.2(c)
-----------------------------------


In [17]:
result = recom.getMovies(title='아이언맨 2')

In [19]:
result['title']

0                     아이언맨 3
1                       아이언맨
2                        로보캅
4                       어벤져스
3                        앤트맨
5                 스파이더맨: 홈커밍
203             스타 트렉: 더 비기닝
457     스타워즈 에피소드 3 - 시스의 복수
1598                    아일랜드
187            매트릭스 2 - 리로디드
Name: title, dtype: object

In [20]:
recom.getMovies(title='어벤져스')

,key_0,key_00,Unnamed: 0,title,genre,year,date,rating,vote_count,plot,...,img_url,keywords,plot_preprocessed_kkma,vec_sim,weighted_vote,genre_sim,vec_sim_scaled,wvote_scaled,genre_scaled,weighted_sum
0,9880,9880,9883,핸콕,액션|판타지|코미디|드라마,2008,7.02,7.36,4329,"X맨, 슈퍼맨, 배트맨, 스파이더맨이 가지고 있던 모든 능력에 누구도 건드릴 수 없...",...,https://movie-phinf.pstatic.net/20111223_45/13...,"['히어로', '수록', '겸비', '스미스', '비밀', '자신', '탄생', '...",맨 슈퍼맨 배트 맨 스파이 더 맨 가지 모든 능력 누 구도 건드리 독특 성격 겸비 ...,1,7.400390,0.169031,1.000000,0.714700,0.169031,0.776746
1,8931,8931,8933,킥 애스: 영웅의 탄생,액션|드라마,2010,4.22,8.47,3554,"지금, 세상은 영웅이 필요한데 ‘왜 아무도 슈퍼히어로가 되려고 하지 않는가?’라는 ...",...,https://movie-phinf.pstatic.net/20111223_23/13...,"['데이브', '수호', '마약', '디', '세상', '영웅', '필요', '민'...",지금 세상 영웅 필요 슈퍼 히 어로 되 하 의문 가지 데 이브 정의 수호 위하 닉네...,0.868801,8.313838,0.258199,0.868801,0.845285,0.258199,0.741978
3,1917,1917,1917,아쿠아맨,액션|모험|SF,2018,12.19,8.36,19729,"땅의 아들이자 바다의 왕, 심해의 수호자인 슈퍼히어로 아쿠아맨의 탄생을 그린 액션 ...",...,https://movie-phinf.pstatic.net/20181210_126/1...,"['바다', '탄생', '심해', '슈퍼히어로', '액션', '블록버스터', '땅'...",땅 아들 자 바다 왕 심해 수호자 슈퍼 히 어로 아쿠아 맨 탄생 그리 액션 블록버스터\n,0.733689,8.331167,0.600000,0.733689,0.847763,0.600000,0.729766
2,6523,6523,6524,변태 가면,액션|코미디,2013,11.14,7.44,234,헐리웃 슈퍼히어로보다 인간적이고 내면에 내재된 강력한 파워의 소유자! 민완 형사인 ...,...,https://movie-phinf.pstatic.net/20131111_227/1...,"['슈퍼히어로', '학교', '귀가', '징벌', '본인', '몸', '여자', '...",헐 슈퍼 히 어로 인간적 내면 내재 파워 소유자 민완 형 사인 아빠 엄마 사이 태어...,0.821543,7.578623,0.258199,0.821543,0.740180,0.258199,0.692602
7,4457,4457,4458,빅 히어로,애니메이션|액션|코미디|가족|SF,2015,1.21,9.12,10731,천재 공학도 ‘테디’가 개발한 힐링로봇 ‘베이맥스’! ‘테디’의 동생이자 로봇 전문...,...,https://movie-phinf.pstatic.net/20150121_287/1...,"['로봇', '전문가', '개발', '힐링', '동생', '도시', '위기', '파...",천재 공학도 개발 힐링 로봇 동생 이자 로봇 전문가 도시 파괴 위기 처하 슈퍼 히 ...,0.699351,9.016000,0.298142,0.699351,0.945665,0.298142,0.668372
9,3128,3128,3128,파워레인져스: 더 비기닝,액션|모험|판타지|SF,2017,4.20,7.21,1135,"정체불명의 우주선을 발견한 뒤, 알 수 없는 강력한 힘을 얻게 된 제이슨, 킴벌리,...",...,https://movie-phinf.pstatic.net/20170420_160/1...,"['트리', '우주선', '발견', '킴벌리', '빌리', '절대', '파멸', '...",정체불명 우주선 발견 뒤 알 힘 얻 되 제이슨 킴벌리 비 트리 잭 세상 파멸 절대 ...,0.609726,7.379063,0.507093,0.609726,0.711652,0.507093,0.609585
5,1029,1029,1029,샤잠!,액션|판타지|SF,2019,4.03,5.81,4973,"솔로몬의 지혜, 헤라클레스의 힘, 아틀라스의 체력, 제우스의 권능, 아킬레스의 용기...",...,https://movie-phinf.pstatic.net/20190322_198/1...,"['스피드', '능력', '악당', '힘', '지혜', '헤라클레스', '체력', ...",솔로몬 지혜 헤라클레스 힘 아틀라스 체력 제우스 권능 아킬레스 용기 머큐리 스피드 ...,0.699989,6.060445,0.400000,0.699989,0.523145,0.400000,0.604622
13,11873,11873,11877,젠틀맨 리그,액션|SF|판타지|모험,2003,8.14,7.32,657,"{1899년 유럽의 강대국들은 아주 불안한 평화상태를 유지했다. 수백년 동안, 전쟁...",...,https://movie-phinf.pstatic.net/20111222_151/1...,"['마스터', '헌터', '평화', '전', '캡틴', '전쟁', '투명인간', '...",유럽 강대국 불안 평화 상태 유지 동안 전쟁 무기 가지 싸우 단발 장총 기병 말 끄...,0.50664,7.484202,0.676123,0.506640,0.726682,0.676123,0.584545
10,892,892,892,마이펫의 이중생활2,애니메이션|모험|코미디,2019,7.31,8.76,1965,집을 비우면 다시 시작되는 펫들의 시크릿 라이프! 집구석 걱정에 하루도 편할 날이 ...,...,https://movie-phinf.pstatic.net/20190805_203/1...,"['고양이', '시작', '하루', '영혼', '슈퍼히어로', '속마음', '공개'...",집 비우 시작 시크 라이프 집구석 걱정 하루 날 사랑 자유 영혼 슈퍼 히 어로 따르...,0.600823,8.430231,0.200000,0.600823,0.861924,0.200000,0.572879
12,8457,8457,8459,퍼스트 어벤져,액션|모험,2011,7.28,7.79,4162,"포기를 모르는 자의 새로운 이름 ‘퍼스트 어벤져’ 세계 전쟁으로 암흑에 빠진 시기,...",...,https://movie-phinf.pstatic.net/20111223_44/13...,"['인류', '최후', '양성', '남', '최고', '초월', '인간', '한계'...",포기 모르 자의 새 이름 세계 전쟁 암흑 빠지 시기 남자 군 입대 자원 이름 남 마...,0.524791,7.762495,0.516398,0.524791,0.766466,0.516398,0.571448
